In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from utils import load_cv_file
from pydantic import BaseModel, Field

In [7]:
client = ChatOpenAI(temperature=0, model="gpt-4o")

In [ ]:
class CVDetails(BaseModel):
    cv_summary :str = Field(description="The overall summary of the person Cv")

In [ ]:
from bs4 import BeautifulSoup
import requests

html_text = requests.get("https://www.myjob.mu/Jobs/ICT-IT-Web/").text

soup = BeautifulSoup(html_text, "lxml")
job_results = soup.find_all("div", class_="module job-result")
if len(job_results)> 0:
    for job in job_results:
        company_name = job.find('h3', itemprop="name")
        company_name = company_name.a.text
        print(f"company_name is {company_name}")
        job_name = job.find("h2", itemprop="title")
        job_name = job_name.a.text
        job_results_overview = job.find("div", class_="job-result-overview")
        overview = job_results_overview.find("")
        print(f"job name is: {job_name}")
        #job_description =



company_name is Alentaris Recruitment Ltd
job name is: IT Coordinator (2504 ITC)
company_name is EURO CRM (Mauritius) Ltd
job name is: TÉLÉVENDEUR DÉBUTANT (VENTE ENTRANTE) SITE EBENE
company_name is EURO CRM (Mauritius) Ltd
job name is: TÉLÉVENDEUR AVEC OU SANS EXPÉRIENCE DU LUNDI-VENDREDI
company_name is Hidden Company
job name is: DevOps Engineer
company_name is ELCA
job name is: Associate Engineer - Power Platform
company_name is ELCA
job name is: Associate Engineer Business Central
company_name is TALENTEUM
job name is: Technicien de Support S.I. Retail
company_name is TALENTEUM
job name is: Technicien de Support Billingue
company_name is TALENTEUM
job name is: Chief Technology Officer (CTO)
company_name is TALENTEUM
job name is: Développeur C#
company_name is TALENTEUM
job name is: Développeur Expérimenté PL/SQL – Opportunité d'Expatriation à Maurice
company_name is TALENTEUM
job name is: Ingénieur Cloud & DevOps (Full Remote, Contrat Indépendant)
company_name is TALENTEUM
job na